In [ ]:
!pwd

/content


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/
!git clone --quiet --recursive https://github.com/amilworks/Hierarchical-Localization.git/


Mounted at /content/drive
/content/drive/MyDrive


In [2]:
%cd /content/drive/MyDrive/Hierarchical-Localization
%load_ext autoreload
%autoreload 2

/content/drive/MyDrive/Hierarchical-Localization


In [3]:

!pip install --progress-bar off --quiet -e .
!pip install --progress-bar off --quiet --upgrade plotly
!pip install opencv-python

  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
import tqdm, tqdm.notebook

tqdm.tqdm = tqdm.notebook.tqdm  # notebook-friendly progress bars
from pathlib import Path
import numpy as np

from hloc import (
    extract_features,
    match_features,
    reconstruction,
    visualization,
    pairs_from_exhaustive,
)
from hloc.visualization import plot_images, read_image
from hloc.utils import viz_3d


In [ ]:
#paths to update
images = Path("datasets")
outputs = Path("outputs/demo-front/")
!rm -rf $outputs
sfm_pairs = outputs / "pairs-sfm.txt"
loc_pairs = outputs / "pairs-loc.txt"
sfm_dir = outputs / "sfm"
features = outputs / "features.h5"
matches = outputs / "matches.h5"
db_out = outputs / "database.db"
#feature_conf = extract_features.confs["superpoint_max"]
#matcher_conf = match_features.confs["superpoint+lightglue"]
matches_file = matches
features_file = features
image_dir = images
feature_conf = extract_features.confs["sift"]
matcher_conf = match_features.confs["adalam"]



In [7]:
# We simply gather all the file paths and convert them into a form that is easier to handle
references = [p.relative_to(images).as_posix() for p in images.iterdir()]
print(len(references), "mapping images")



104 mapping images


In [8]:
extract_features.main(
    feature_conf, images, image_list=references, feature_path=features
)
pairs_from_exhaustive.main(sfm_pairs, image_list=references)
match_features.main(matcher_conf, sfm_pairs, features=features, matches=matches);

[2025/05/10 01:37:47 hloc INFO] Extracting local features with configuration:
{'model': {'name': 'dog'},
 'output': 'feats-sift',
 'preprocessing': {'grayscale': True, 'resize_max': 1600}}


  0%|          | 0/104 [00:00<?, ?it/s]

[2025/05/10 01:40:08 hloc INFO] Finished exporting features.
[2025/05/10 01:40:08 hloc INFO] Found 5356 pairs.
[2025/05/10 01:40:08 hloc INFO] Matching local features with configuration:
{'model': {'name': 'adalam'}, 'output': 'matches-adalam'}
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


  0%|          | 0/5356 [00:00<?, ?it/s]

[2025/05/10 02:49:39 hloc INFO] Finished exporting matches.


In [9]:

model = reconstruction.main(
    sfm_dir, images, sfm_pairs, features, matches, image_list=references, verbose=True
)

fig = viz_3d.init_figure()
viz_3d.plot_reconstruction(
    fig, model, color="rgba(255,0,0,0.5)", name="mapping", points_rgb=True
)
fig.show()
### for second match features

from hloc.utils.read_write_model import read_model, write_model


# Read the binary model
cameras, images, points3D = read_model(sfm_dir, ext=".bin")

# Write the text model
write_model(cameras, images, points3D, sfm_dir, ext=".txt")

[2025/05/10 03:16:30 hloc INFO] Creating an empty database...
[2025/05/10 03:16:30 hloc INFO] Importing images into the database...
[2025/05/10 03:16:49 hloc INFO] Importing features into the database...


  0%|          | 0/104 [00:00<?, ?it/s]

[2025/05/10 03:16:50 hloc INFO] Importing matches into the database...


  0%|          | 0/5356 [00:00<?, ?it/s]

[2025/05/10 03:18:32 hloc INFO] Performing geometric verification of the matches...
[2025/05/10 03:24:14 hloc INFO] Running 3D reconstruction...
[2025/05/10 03:24:51 hloc INFO] Reconstructed 3 model(s).
[2025/05/10 03:24:51 hloc INFO] Largest model is #2 with 27 images.
[2025/05/10 03:24:51 hloc INFO] Reconstruction statistics:
Reconstruction:
	num_cameras = 1
	num_images = 27
	num_reg_images = 27
	num_points3D = 2432
	num_observations = 8308
	mean_track_length = 3.41612
	mean_observations_per_image = 307.704
	mean_reprojection_error = 1.25835
	num_input_images = 104


({2: Camera(id=2, model='SIMPLE_RADIAL', width=4032, height=3024, params=array([6.11704902e+03, 2.01600000e+03, 1.51200000e+03, 1.32942268e-02]))},
 {58: Image(id=58, qvec=array([ 0.99709281,  0.07214389,  0.02259167, -0.00952864]), tvec=array([-2.9316862 ,  1.1892566 , -3.87180629]), camera_id=2, name='IMG_1497.JPG', xys=array([[1026.      ,   10.75    ],
         [1064.      ,   10.484375],
         [ 498.5     ,   11.71875 ],
         ...,
         [2374.      , 2442.      ],
         [3630.      , 2556.      ],
         [ 347.5     , 2732.      ]]), point3D_ids=array([-1, -1, -1, ..., -1, -1, -1])),
  64: Image(id=64, qvec=array([ 0.99851152,  0.01210873,  0.05212812, -0.01052579]), tvec=array([ 3.16768009, -0.7147509 ,  3.80196444]), camera_id=2, name='IMG_1521.JPG', xys=array([[2694.       ,    7.328125 ],
         [3798.       ,   11.7578125],
         [2512.       ,   15.71875  ],
         ...,
         [1208.       , 1988.       ],
         [1914.       , 2348.       ],
      